Install imageio for image related operations and opencv for video related operations.

In [1]:
!pip install -q imageio
!pip install -q opencv-python
!pip install -q git+https://github.com/tensorflow/docs

Import neccessary modules 

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import imageio
from urllib import request
import cv2
import os

Loading videos' ulrs for importing and processing them to feed into pre-trained model.

In [3]:
urls_kd = []
urls_kk = []

In [4]:
d = "/content/drive/My Drive/unn/normal/khabaddi"
for path in os.listdir(d):
  
  full_path = os.path.join(d, path)
  if os.path.isfile(full_path):
    urls_kd.append(full_path)

In [ ]:
urls_kd

In [5]:
d = "/content/drive/My Drive/unn/normal/kho kho"
for path in os.listdir(d):
  
  full_path = os.path.join(d, path)
  if os.path.isfile(full_path):
    urls_kk.append(full_path)

In [ ]:
urls_kk

Processing videos, getting images from frames using cv module

In [6]:
def load_video(path, max_frames=0, resize=(224, 224)):
  cap = cv2.VideoCapture(path)
  frames = []
  try:
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      frame = cv2.resize(frame, resize)
      frame = frame[:, :, [2, 1, 0]]
      frames.append(frame)
      
      if len(frames) == max_frames:
        break
  finally:
    cap.release()
    len(frames)
  return (np.array(frames) / 255.0)
  

Importing labels and pretrained model from tensorflow hub

In [7]:
# Get the kinetics-400 action labels from the GitHub repository.
KINETICS_URL = "https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map_600.txt"
with request.urlopen(KINETICS_URL) as obj:
  labels = [line.decode("utf-8").strip() for line in obj.readlines()]
print("Found %d labels." % len(labels))

Found 600 labels.


Loading pre-trained model

In [8]:
i3d = hub.load("https://tfhub.dev/deepmind/i3d-kinetics-600/1").signatures['default']

Predicting video category as per pretrained model. Output of this model is input to the classification model.

In [11]:
def predict(sample_video):
  # Add a batch axis to the to the sample video.
  model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

  logits = i3d(model_input)['default'][0]
  tf.nn.softmax(logits)
  
  return tf.nn.softmax(logits)

In [12]:
probabilities_kd = []
probabilities_kk = []

In [ ]:
for i in urls_kd:
  print(i)
  v = []
  v = load_video(i)
  predict(v)
  probabilities_kd.append(predict(v))
  print(probabilities_kd[-1])

In [ ]:
for i in urls_kk:
  print(i)
  v = []
  v = load_video(i)
  predict(v)
  probabilities_kk.append(predict(v))
  print(probabilities_kk[-1])

In [15]:
probabilities_kd = np.array(probabilities_kd)
probabilities_kk = np.array(probabilities_kk)

In [17]:
probabilities_kd.shape

(17, 600)

Adding labels info to the intermediatory output and converting to pandas dataframe and shuffling it to feed into classification model.

In [18]:
z = np.zeros((17,1)) #considering khabaddi videos to be of 0-cateogory

In [19]:
z. shape

(17, 1)

In [20]:
pp = probabilities_kd
pp.shape

(17, 600)

In [21]:
probabilities_kd = np.append(probabilities_kd, z, axis=1)
probabilities_kd.shape

(17, 601)

In [22]:
pk = probabilities_kk
pk.shape

(13, 600)

In [127]:
zk = np.ones((13,1)) #considering kho kho videos to be of 1-cateogory

In [24]:
probabilities_kk = np.append(probabilities_kk, zk, axis=1)
probabilities_kk.shape

(13, 601)

In [26]:
probs = np.concatenate((probabilities_kd, probabilities_kk), axis=0)
probs.shape

(30, 601)

In [96]:
df = pd.DataFrame(data=prob)
df.tail()  #Converting to pandas dataframe to load data to model

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600
25,7.725971e-05,0.000033,2.595261e-07,8.162342e-06,3.526550e-07,4.953244e-05,1.393781e-04,0.000028,2.117869e-07,0.000111,2.453876e-06,9.535331e-06,4.906322e-08,3.575907e-07,1.148394e-07,1.709181e-04,1.517883e-07,0.000022,2.520223e-07,8.346986e-08,3.082698e-07,5.465634e-05,1.197984e-06,3.393456e-07,0.000033,1.838872e-05,1.248896e-07,1.095218e-04,3.132707e-07,0.000096,2.501920e-07,4.426642e-08,8.857810e-07,1.420389e-08,1.098550e-07,2.573577e-07,9.989686e-07,8.245372e-07,8.741645e-08,6.795821e-07,...,2.441661e-05,7.549850e-07,1.285083e-05,3.914981e-07,1.694390e-08,9.571919e-08,0.000004,4.733000e-07,1.649238e-07,2.254075e-07,1.509203e-06,3.644812e-07,3.115687e-08,3.724071e-08,1.197536e-07,6.564261e-07,1.933979e-05,4.993186e-06,4.776703e-07,8.432774e-08,7.195490e-06,1.476732e-08,1.152608e-08,6.011177e-09,4.049332e-08,2.656289e-09,5.363825e-09,1.203507e-06,1.252002e-06,3.194476e-05,1.273062e-05,1.843039e-08,9.809206e-08,0.000306,6.883828e-07,1.666949e-09,5.792581e-07,0.000213,0.052899,0.0
26,1.885309e-04,0.000152,1.378803e-06,1.846628e-05,1.341620e-06,1.482278e-04,2.784087e-04,0.000091,1.230250e-06,0.000199,3.632555e-06,1.644047e-05,1.655193e-07,1.295115e-06,1.840079e-07,9.309063e-04,7.464193e-07,0.000028,8.733620e-07,1.046094e-06,7.530790e-07,6.560834e-05,2.084701e-06,2.021401e-06,0.000064,4.250009e-05,3.541051e-08,2.870668e-04,5.486384e-07,0.000181,7.823579e-07,5.086818e-07,1.090756e-06,6.084000e-08,3.729373e-07,2.077686e-06,2.343930e-06,2.570716e-06,3.849793e-07,4.513779e-06,...,2.589511e-05,6.675483e-06,1.807343e-05,3.513726e-06,1.196361e-07,4.164961e-07,0.000024,8.023975e-06,9.026637e-07,1.009552e-06,1.711453e-05,3.072136e-06,1.354532e-07,2.901521e-07,6.979709e-07,1.971597e-06,2.420202e-05,9.637985e-06,6.042645e-06,2.047634e-07,1.260829e-05,1.327651e-07,5.024073e-08,2.944371e-08,1.984181e-07,1.916648e-08,4.240396e-08,2.864517e-06,2.247076e-06,1.281046e-04,4.601285e-05,6.897311e-08,7.848599e-07,0.000425,2.521528e-06,1.740839e-08,2.884530e-06,0.000329,0.060368,0.0
27,6.571332e-07,0.000204,2.257266e-06,2.906417e-06,2.016336e-05,1.202370e-05,4.681996e-05,0.000160,2.025838e-07,0.000108,8.806151e-06,1.585945e-05,1.601449e-07,1.110520e-06,4.931234e-07,2.154173e-05,2.117622e-06,0.001512,3.565430e-07,7.603892e-07,9.213034e-06,1.039070e-06,5.418861e-07,7.088097e-06,0.000243,3.241246e-06,4.556925e-10,2.650358e-05,3.295703e-06,0.000112,2.167009e-08,1.513974e-05,6.507115e-08,8.733975e-07,5.341643e-06,4.341153e-07,5.290732e-06,1.047365e-06,1.160946e-06,3.434186e-06,...,5.438952e-06,1.401887e-06,1.845089e-05,7.961750e-06,1.557580e-06,2.765351e-05,0.000092,7.290747e-05,5.606350e-06,6.291791e-06,4.574113e-05,3.837520e-05,2.511706e-06,2.093228e-06,1.862425e-05,1.743865e-06,3.694930e-05,4.915927e-05,1.097061e-03,2.250581e-07,2.505738e-05,5.970452e-07,4.120377e-07,1.382681e-07,1.790051e-06,2.865190e-07,4.258146e-07,3.496415e-07,2.868650e-06,6.179098e-04,1.223281e-05,3.461168e-07,5.082791e-07,0.000150,5.728064e-06,8.762611e-08,5.822356e-05,0.002554,0.000168,1.0
28,2.479078e-08,0.000003,5.661215e-09,2.977107e-08,1.185825e-09,1.727938e-08,2.856995e-07,0.000013,2.118010e-08,0.000001,3.835580e-08,6.752188e-08,2.530587e-09,2.306924e-08,7.076283e-09,3.110405e-07,4.983176e-09,0.000010,1.863026e-08,2.446786e-09,6.062073e-08,1.447140e-08,1.685439e-08,1.503592e-07,0.000009,4.500132e-08,5.270628e-12,6.163621e-07,3.933244e-09,0.000001,1.004209e-09,7.240928e-09,2.079663e-09,1.134916e-08,2.955545e-08,7.519109e-10,1.952617e-08,5.574607e-08,2.597049e-09,2.065926e-08,...,2.550333e-07,7.420012e-09,1.723888e-07,2.041391e-07,9.917281e-09,2.704913e-08,0.000037,1.486291e-07,5.170145e-08,3.162640e-07,1.923901e-07,1.830236e-08,3.616335e-09,4.698919e-09,1.032360e-07,4.396121e-08,6.757184e-07,6.361582e-0

In [97]:
target = df.pop(df.columns[-1])
target #Target or labels seperation from feature set.

0     1.0
1     0.0
2     1.0
3     1.0
4     1.0
5     0.0
6     1.0
7     0.0
8     0.0
9     0.0
10    1.0
11    0.0
12    0.0
13    1.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    1.0
22    1.0
23    1.0
24    1.0
25    0.0
26    0.0
27    1.0
28    1.0
29    0.0
Name: 600, dtype: float64

In [102]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [103]:
train_dataset = dataset.shuffle(len(df)).batch(1)

Defining classification model and compling it.

In [113]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(700, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dropout(0.15),
    tf.keras.layers.Dense(350, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
  ])

  model.compile(optimizer='RMSprop',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [114]:
model = get_compiled_model()
model.fit(train_dataset, epochs=44)

Epoch 1/44

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



30/30 [==============================] - 0s 9ms/step - loss: 0.7064 - accuracy: 0.5667
Epoch 2/44
30/30 [==============================] - 0s 9ms/step - loss: 0.6162 - accuracy: 0.6333
Epoch 3/44
30/30 [==============================] - 0s 8ms/step - loss: 0.4775 - accuracy: 0.8667
Epoch 4/44
30/30 [==============================] - 0s 10ms/step - loss: 0.3757 - accuracy: 0.9667
Epoch 5/44
30/30 [==============================] - 0s 8ms/step - loss: 0.3586 - accuracy: 0.9667
Epoch 6/44
30/30 [==============================] - 0s 9ms/step - loss: 0.3514 - accuracy: 0.9667
Epoch 7/44
30/30 [==============================] - 0s 9ms/step - loss: 0.3467 - accuracy: 0.9667
Epoch 8/44
30/30 [==============================] - 0s 9ms/step - loss: 0.3467 - accuracy: 0.9667
Epoch 9/44
30/30 [==============================] - 0s 9ms/step - loss: 0.3148 - accuracy: 1.0000
Epoch 10/44
30/30 [==============================] - 0s 8ms/step - loss: 0.3927 - accuracy: 0.9000
Epoch 11/44
30/30 [==========

Testing out the classifier with kho kho and khabaddi test videos 

In [115]:
sample = load_video("/content/kk-t.mp4") #Loading test video of kho kho to test the model

In [41]:
sam = predict(sample)

In [ ]:
sam

Adding batch dimenssion to the input for the classifier model.

In [55]:
model_input = tf.constant(sam, dtype=tf.float32)[tf.newaxis,...]

In [116]:
prediction = model(model_input)

Model correclty classified video to be belonging to kho kho video as it gave output of 1-category being highest probability.

In [117]:
prediction

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[3.1358738e-08, 1.0000000e+00]], dtype=float32)>

In [120]:
sample1 = load_video("/content/Untitled.mp4") #Loading test video of khabaddi video to test the model

In [121]:
sam1=predict(sample1)

In [122]:
model_input1 = tf.constant(sam1, dtype=tf.float32)[tf.newaxis, ...]

In [123]:
prediction1 = model(model_input1)

Model correclty classified video to be beloging to kho kho video as it gave output of 0-category being highest probability.

In [124]:
prediction1

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[1.0000000e+00, 1.4039028e-23]], dtype=float32)>

In [126]:
model.save("/content")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /content/assets


INFO:tensorflow:Assets written to: /content/assets
